# Soccer Dream Team Selection Problem
### Data Pre-Processing and Exploration

What makes a successful soccer dream team?

This is not a simple question and the answer probably depends on many factors and variables. Some may be as logical as the individual performance of the players, but this does not ensure the success of the team. 
A clear example of the above is the Paris Saint German squad, which despite being one of the most expensive teams and with the best players, has not yet achieved a Champions League championship.

There can probably be other important factors such as the connection and communication that the players have with each other, the strategies used by the coach and even financial constraints. Despite the above, the proposed optimization model focuses on a quantitative analysis that chooses the players with the best individual performance to maximize the overall performance of the team subject to multiple constraints.

In summary, the optimization model is subject to the following constraints:

Regulatory and budget Constraints:
- Select 23 players (11 initial players and 12 sustitute players)
- Number of non european union players <= 5, based on the LaLiga requirements
- Option to choose a limit budget from three available options:
1. Low_budget = EUR 50,887,000 - squad value of Real Zaragoza (lowest squad value in LaLiga)
2. Medium_budget = EUR 581,728,000 - squad value of Sevilla (mid squad value in LaLiga)
3. High_budget = EUR 1,171,276,000 - squad value of Real Madrid (highest squad value in LaLiga)

Strategic Constraints:
- At least one tall defense with a height of at least 186 cm (75 percentile) to defend the aerial plays or corner kicks of the rival team
- Option to choose a formation from three available options (4-4-2, 4-3-3, 3-5-2)
- To have at least 2 but not more that 4 experienced players (29 years old that represents 75% percentile) so they can lead the most unexperienced ones in important matches
- Have at least 4 players that meet the criteria of potential growth (see more detail through the code), to have players that don't have the best score currently but they have a bright future in the next years and a great value to bring to the team at a relatively low price
- From the 2 Goal Keepers, have at least 1 that meets the criteria of potential growth

In [1]:
import gurobipy as gb # for optimization
from gurobipy import * # for optimization
import numpy as np #for arrays and statistics
import pandas as pd #for data pre-processing

The dataset used was retrieved from Kaggle. It is made up of one excel file that contains the main attributes of all players in the FIFA 2022 game developed by Electronic Arts.

In [2]:
db = pd.read_excel('players_22_final.xlsx', sheet_name='players_22_final')
db.dropna(inplace=True)
display(db.head())
positions = db["standard_position"]

,sofifa_id,short_name,standard_position,score,value_eur,annual_wage_eur,potential,age,club_contract_valid_until,release_clause_eur,nationality_name,European_Union,height_cm
0,158023,L. Messi,RW (Right Wing),93,78000000.0,16640000.0,93,34,2023.0,144300000.0,Argentina,0,170
1,188545,R. Lewandowski,CF (Center Forward),92,119500000.0,14040000.0,92,32,2023.0,197200000.0,Poland,1,185
2,20801,Cristiano Ronaldo,CF (Center Forward),91,45000000.0,14040000.0,91,36,2023.0,83300000.0,Portugal,1,187
3,190871,Neymar Jr,LW (Left Wing),91,129000000.0,14040000.0,91,29,2025.0,238700000.0,Brazil,0,175
4,192985,K. De Bruyne,CM (Center Midfielder),91,125500000.0,18200000.0,91,30,2025.0,232200000.0,Belgium,1,181


For the data pre-processing, the dummies for the standard position were created to have auxiliary binaries for the formation and position that is to be optimized.

In [3]:
#Dummyfying the positions
positions = db["standard_position"]
db = pd.get_dummies(data=db, columns=["standard_position"], prefix="")

To have a succesful team now and in the future, players with potential to grow should also be considered as part of the dream team. Therefore through data analysis and statistics the goal is to identify players that comply with this.

For players that have a current score equal or less than 70, the potential to grow should be 10 points that represents the percentile 75 in players below 70 score. Ex. a player that currently has 70 should have at least a potential of 80 to meet this constraint. 

For players that have a current score from 71 to 80, the potential to grow should be 4 points that represents the percentile 75 in players between 71-80. Ex. a player that currently has 75 should have at least a potential of 79 to meet this constraint. 

For players that have a current score from 81 to 90, the potential to grow should be 3 points that represents the percentile 75 in players between 81-90. Ex. a player that currently has 85 should have at least a potential of 88 to meet this constraint. 

Players above 90, don't need to meet a grow potential since they no longer have much room to continue growing.

In [4]:
#Creating auxiliary binary to see potential of growth
db["Potential_Growth"] = db["potential"]-db["score"]

In [5]:
#Creating auxiliary binary to see potential of growth
# Put as 1, players between 0-70 of score with a potential to grow up at least 10 points. Otherwise, put zero.
# Put as 1, players between 70-80 of score with a potential to grow up at least 4 points. Otherwise, put zero.
# Put as 1, players between 80-90 of score with a potential to grow up at least 3 points. Otherwise, put zero.

binary_potential = []

for i in range(len(db)):
    if db.iloc[i,2] <= 70 and db.iloc[i,22] >= 10: # The 10 points are the 75 percentile of players <70 score 
        binary_potential.append(1)
    elif db.iloc[i,2] <= 80 and db.iloc[i,22] >= 4:# The 4 points are the 75 percentile of players 71-80 score
        binary_potential.append(1)
    elif db.iloc[i,2] <= 90 and db.iloc[i,22] >= 3:# The 3 points are the 75 percentile of players 81-90 score 
        binary_potential.append(1)
    else:
        binary_potential.append(0)
    
db["Binary_Potential"] = binary_potential

In [6]:
#Creating auxiliary binary to see if player is older than 29 years (75 percentile)
print("The 75 percentile in the attribute age is:",np.percentile(np.array(db)[:,6],75))
db["Binary_Age"] = (db["age"]>=np.percentile(np.array(db)[:,6],75)).astype(int)
db.head()

The 75 percentile in the attribute age is: 29.0


,sofifa_id,short_name,score,value_eur,annual_wage_eur,potential,age,club_contract_valid_until,release_clause_eur,nationality_name,...,_GK (Goal Keeper),_LB (Left Back),_LM (Left Midfielder),_LW (Left Wing),_RB (Right Back),_RM (Right Midfielder),_RW (Right Wing),Potential_Growth,Binary_Potential,Binary_Age
0,158023,L. Messi,93,78000000.0,16640000.0,93,34,2023.0,144300000.0,Argentina,...,0,0,0,0,0,0,1,0,0,1
1,188545,R. Lewandowski,92,119500000.0,14040000.0,92,32,2023.0,197200000.0,Poland,...,0,0,0,0,0,0,0,0,0,1
2,20801,Cristiano Ronaldo,91,45000000.0,14040000.0,91,36,2023.0,83300000.0,Portugal,...,0,0,0,0,0,0,0,0,0,1
3,190871,Neymar Jr,91,129000000.0,14040000.0,91,29,2025.0,238700000.0,Brazil,...,0,0,0,1,0,0,0,0,0,1
4,192985,K. De Bruyne,91,125500000.0,18200000.0,91,30,2025.0,232200000.0,Belgium,...,0,0,0,0,0,0,0,0,0,1


In [7]:
db["Standard_Position"] = positions
db_array = np.array(db)

In [8]:
sofifa = db_array[:,0]
name = db_array[:,1]
score = db_array[:,2]
value = db_array[:,3]
wage = db_array[:,4]
potential = db_array[:,5]
age = db_array[:,6]
contract = db_array[:,7]
release_amount = db_array[:,8]
nationality = db_array[:,9]
europe = db_array[:,10]
height = db_array[:,11]
cb = db_array[:,12]
cf = db_array[:,13]
cm = db_array[:,14]
gk = db_array[:,15]
lb = db_array[:,16]
lm = db_array[:,17]
lw = db_array[:,18]
rb = db_array[:,19]
rm = db_array[:,20]
rw = db_array[:,21]
aux_potential = db_array[:,23]
aux_age = db_array[:,24]
positions = db_array[:,25]

### Optimization

In [9]:
model = gb.Model("Dream Team Selection Problem")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [10]:
n = len(name)
print("The total number of players in the database is","{:,}".format(n))

The total number of players in the database is 18,063


As the dataset has 18,063 players, the optimization problem has the same number of variables to decide whether the player is chosen.

Additionally, it has three decision variables that serve as auxiliary binaries to build the constraint of a tall defense.

In [11]:
# Decision variables
X = model.addVars(n,vtype=GRB.BINARY,name=[i for i in name])
b_lb = model.addVars(n,vtype=GRB.BINARY,name=["b_lf: "+i for i in name])
b_rb = model.addVars(n,vtype=GRB.BINARY,name=["b_rb: "+i for i in name])
b_cb = model.addVars(n,vtype=GRB.BINARY,name=["b_cb: "+i for i in name])

The optimization problem is maximizing the sum of the product of the score of chosen players times the binary variable which determines if the player was selected or not.

In [12]:
# Objective function is to maximize the average squad score
model.setObjective(sum(X[i]*score[i]/23 for i in range (n)),GRB.MAXIMIZE)

### Regulatory and Budget Constraints

According to La Liga (Spain), in each match the team can take 18 players and have 3 changes of players. However, each team usually has around 23 players registered to give opportunities to new ones and give rest to others. Therefore the optimization problem will target 23 players.

In [13]:
# Constraint of twenty-three players
model.addConstr(sum(X[i] for i in range (n)) == 23)

<gurobi.Constr *Awaiting Model Update*>

In [14]:
# Constraint of at least 18 european union players (European Union rule)
model.addConstr(sum(X[i]*europe[i] for i in range(n))>=18)

<gurobi.Constr *Awaiting Model Update*>

Constraint of budget

For this constraint, we will take the idea that a shareholder organization that is dedicated to invest in soccer clubs has the sufficient budget to buy a LaLiga team. The organization wants our advice to know if with that same budget it is better to buy players directly in the market and create a new club from scratch.

For this budget, we are only considering the squad market value of the 23 players of three different teams that played more matches and minutes. In addition to the market value, we are adding the annual wage of this 23 players into the budget.

It is assumed that we don't have other initial investments at creating a new soccer club.

In [15]:
Low_budget = 50887000 #Squad value of Real Zaragoza team (lowest squad value in LaLiga)
Medium_budget = 581728000 #Squad value of Sevilla team
High_budget = 1171276000 #Squad value of Real Madrid (highest squad value in LaLiga)

In [16]:
response_1 = input("Select the letter, based on the budget you want: \na. Low_budget \nb. Medium_budget \nc. High_budget \nAnswer: ")

Select the letter, based on the budget you want: 
a. Low_budget 
b. Medium_budget 
c. High_budget 
Answer: c


In [17]:
if response_1 == "a":
    model.addConstr(sum(X[i]*(wage[i]+value[i]) for i in range(n))<=Low_budget)
elif response_1 == "b":
    model.addConstr(sum(X[i]*(wage[i]+value[i]) for i in range(n))<=Medium_budget)
elif response_1 == "c":
    model.addConstr(sum(X[i]*(wage[i]+value[i]) for i in range(n))<=High_budget)

### Strategic Constraints

To defend the aerial plays or corner kicks of the rival team we want at least one tall defense with a heigh of at least 186 cm (75 percentile)

In [18]:
#Constraint of a tall defense (more than 186 cm (75 percentile))
height_p = np.percentile(height,75)
print(" The 75 percentile in the attribute heigh is:",height_p,"cm")

 The 75 percentile in the attribute heigh is: 186.0 cm


In [19]:
M = 10000
for i in range (n):
    model.addConstr(X[i]*lb[i]*height[i]-height_p<=M*b_lb[i])
    model.addConstr(X[i]*lb[i]*height[i]-height_p>= -M*(1-b_lb[i])) 

for i in range (n):
    model.addConstr(X[i]*rb[i]*height[i]-height_p<=M*b_rb[i])
    model.addConstr(X[i]*rb[i]*height[i]-height_p>= -M*(1-b_rb[i])) 
    
for i in range (n):
    model.addConstr(X[i]*cb[i]*height[i]-height_p<=M*b_cb[i])
    model.addConstr(X[i]*cb[i]*height[i]-height_p>= -M*(1-b_cb[i])) 
    
model.addConstr(sum(b_lb[i]+b_rb[i]+b_cb[i] for i in range (n)) >= 1)


<gurobi.Constr *Awaiting Model Update*>

To have a flexible optimization model that can be adapted to different game strategies, the model allows to choose between 3 different formations: 4-4-2, 4-3-3 and 3-5-2

In [20]:
response = input("Select the letter, based on the formation you want: \na. 4-4-2 \nb. 4-3-3 \nc. 3-5-2 \nAnswer: ")

Select the letter, based on the formation you want: 
a. 4-4-2 
b. 4-3-3 
c. 3-5-2 
Answer: a


In [21]:
if response == "a":
    #Positions Defense
    model.addConstr(sum(X[i]*cb[i] for i in range(n))==4) #The team needs 4 Center Back players
    model.addConstr(sum(X[i]*lb[i] for i in range(n))==2) #The team needs 2 Left Back player
    model.addConstr(sum(X[i]*rb[i] for i in range(n))==2) #The team needs 2 Right Back player

    #Positions Midfielder
    model.addConstr(sum(X[i]*cm[i] for i in range(n))==5) #The team needs 5 Center Midfielder player
    model.addConstr(sum(X[i]*lm[i] for i in range(n))==2) #The team needs 2 Left Midfielder player
    model.addConstr(sum(X[i]*rm[i] for i in range(n))==2) #The team needs 2 Right Midfielder player

    #Positions Wings
    model.addConstr(sum(X[i]*lw[i] for i in range(n))==1) #The team needs 1 Left Wing player
    model.addConstr(sum(X[i]*rw[i] for i in range(n))==1) #The team needs 1 Right Wing player

    #Positions Goalkeeper and Forward
    model.addConstr(sum(X[i]*gk[i] for i in range(n))==2) #The team needs 2 Goalkeeper player
    model.addConstr(sum(X[i]*cf[i] for i in range(n))==2) #The team needs 2 Center Forward player
elif response == "b":
    #Positions Defense
    model.addConstr(sum(X[i]*cb[i] for i in range(n))==4) #The team needs 4 Center Back players
    model.addConstr(sum(X[i]*lb[i] for i in range(n))==2) #The team needs 2 Left Back player
    model.addConstr(sum(X[i]*rb[i] for i in range(n))==2) #The team needs 2 Right Back player

    #Positions Midfielder
    model.addConstr(sum(X[i]*cm[i] for i in range(n))==3) #The team needs 3 Center Midfielder player
    model.addConstr(sum(X[i]*lm[i] for i in range(n))==2) #The team needs 2 Left Midfielder player
    model.addConstr(sum(X[i]*rm[i] for i in range(n))==2) #The team needs 2 Right Midfielder player

    #Positions Wings
    model.addConstr(sum(X[i]*lw[i] for i in range(n))==2) #The team needs 2 Left Wing player
    model.addConstr(sum(X[i]*rw[i] for i in range(n))==2) #The team needs 2 Right Wing player

    #Positions Goalkeeper and Forward
    model.addConstr(sum(X[i]*gk[i] for i in range(n))==2) #The team needs 2 Goalkeeper player
    model.addConstr(sum(X[i]*cf[i] for i in range(n))==2) #The team needs 2 Center Forward player
else:
        #Positions Defense
    model.addConstr(sum(X[i]*cb[i] for i in range(n))==2) #The team needs 2 Center Back players
    model.addConstr(sum(X[i]*lb[i] for i in range(n))==2) #The team needs 2 Left Back player
    model.addConstr(sum(X[i]*rb[i] for i in range(n))==2) #The team needs 2 Right Back player

    #Positions Midfielder
    model.addConstr(sum(X[i]*cm[i] for i in range(n))==3) #The team needs 2 Center Midfielder player
    model.addConstr(sum(X[i]*lm[i] for i in range(n))==2) #The team needs 2 Left Midfielder player
    model.addConstr(sum(X[i]*rm[i] for i in range(n))==2) #The team needs 2 Right Midfielder player

    #Positions Wings
    model.addConstr(sum(X[i]*lw[i] for i in range(n))==2) #The team needs 2 Left Wing player
    model.addConstr(sum(X[i]*rw[i] for i in range(n))==2) #The team needs 2 Right Wing player

    #Positions Goalkeeper and Forward
    model.addConstr(sum(X[i]*gk[i] for i in range(n))==2) #The team needs 2 Goalkeeper player
    model.addConstr(sum(X[i]*cf[i] for i in range(n))==4) #The team needs 4 Center Forward player


For the dynamics of the team, it is also important to consider experienced players so they can lead the most unexperienced ones in important matches. Therefore, a constraint to have at least two players but not more than four players older than 29* years old was included.

*29 years is the 75 percentile in the attribute age.

In [22]:
#Constraint to have between 2 and 4 players greater than 29 years old (75% percentile)
model.addConstr(sum(X[i]*aux_age[i] for i in range(n))>=2)
model.addConstr(sum(X[i]*aux_age[i] for i in range(n))<=4)

<gurobi.Constr *Awaiting Model Update*>

We require the model to select at least 4 players that meet the criteria of potential growth, to have players that maybe don't have the best score currently but they have a bright future in the next years and a great value to bring to the team at a relatively low price. We require that one of this four players is a Goal Keeper. 

In [23]:
#Constraint of potential to have at least four players with potential to grow
model.addConstr(sum(X[i]*aux_potential[i] for i in range(n))>=4)

<gurobi.Constr *Awaiting Model Update*>

In [24]:
#Constraint to have a goalkeeper with potential to grow
model.addConstr(sum(X[i]*aux_potential[i]*gk[i] for i in range(n))>=1)

<gurobi.Constr *Awaiting Model Update*>

### Solving the problem

In [25]:
#Solving the problem
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 108396 rows, 72252 columns and 259712 nonzeros
Model fingerprint: 0x6c6483e6
Variable types: 0 continuous, 72252 integer (72252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+08]
  Objective range  [2e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 65.9130435
Presolve removed 108377 rows and 59052 columns
Presolve time: 0.25s
Presolved: 19 rows, 13200 columns, 45149 nonzeros
Found heuristic solution: objective 76.2608696
Variable types: 0 continuous, 13200 integer (11377 binary)

Root relaxation: objective 8.514004e+01, 127 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

In [26]:
print("\nThe max average score at the optimal selection is","{:,}".format(round(model.objVal,1)))

print("\nThe selected players that maximize the objective function are:"+"\n")

players = []

for v in model.getVars():
    if v.x == 1:
        players.append(v.index)
players = players[0:23]

for player_id in players:
    print(db_array[player_id,1],"with score:",db_array[player_id,2], "and position",db_array[player_id,25])


The max average score at the optimal selection is 85.1

The selected players that maximize the objective function are:

L. Messi with score: 93 and position RW (Right Wing)
Cristiano Ronaldo with score: 91 and position CF (Center Forward)
M. Neuer with score: 90 and position GK (Goal Keeper)
Sergio Ramos with score: 88 and position CB (Center Back)
M. Verratti with score: 87 and position CM (Center Midfielder)
A. Hakimi with score: 85 and position RB (Right Back)
Luis Alberto with score: 84 and position CM (Center Midfielder)
L. Digne with score: 84 and position LB (Left Back)
M. Sabitzer with score: 84 and position CM (Center Midfielder)
M. Ginter with score: 84 and position CB (Center Back)
Y. Carrasco with score: 84 and position LM (Left Midfielder)
F. Kostic with score: 84 and position LM (Left Midfielder)
R. Guerreiro with score: 84 and position LB (Left Back)
Ricardo Pereira with score: 84 and position RB (Right Back)
M. Maignan with score: 84 and position GK (Goal Keeper)
M. Br

### Splitting the 23 players between the 11 starting players and 12 sustitute players

In [27]:
selected_team = db.iloc[players,:]
positions=[ 'GK (Goal Keeper)', 'LB (Left Back)', 'CB (Center Back)',
 'RB (Right Back)' , 'LM (Left Midfielder)',  'CM (Center Midfielder)' ,'RM (Right Midfielder)',
 'LW (Left Wing)' ,'RW (Right Wing)','CF (Center Forward)']
#print(selected_team)

form_442 = [1,1,2,1,1,2,1,0,1,1]
form_433 = [1,1,2,1,1,1,1,1,1,1]
form_352 = [1,1,1,1,1,1,1,1,1,2]

starting_11 = []
if response == "a":
    for i in range(len(positions)):
        x = selected_team[selected_team["Standard_Position"]==positions[i]].nlargest(form_442[i], 'score')
        starting_11.extend(list(x.iloc[:,0]))
elif response == "b":
    for i in range(len(positions)):
        x = selected_team[selected_team["Standard_Position"]==positions[i]].nlargest(form_433[i], 'score')
        starting_11.extend(list(x.iloc[:,0]))
else:
    for i in range(len(positions)):
        x = selected_team[selected_team["Standard_Position"]==positions[i]].nlargest(form_352[i], 'score')
        starting_11.extend(list(x.iloc[:,0]))
        
selected_team['Starting_11'] = selected_team['sofifa_id'].isin(starting_11).astype(int)

print("The starting team is: ")
for i in range(len(selected_team)):
    if selected_team.iloc[i,26] == 1:
        print(selected_team.iloc[i,1],"with score:",selected_team.iloc[i,2], "and position", selected_team.iloc[i,25])

print("\nThe substitutes are: ")
for i in range(len(selected_team)):
    if selected_team.iloc[i,26] ==0:
        print(selected_team.iloc[i,1],"with score:",selected_team.iloc[i,2], "and position", selected_team.iloc[i,25])
   

The starting team is: 
L. Messi with score: 93 and position RW (Right Wing)
Cristiano Ronaldo with score: 91 and position CF (Center Forward)
M. Neuer with score: 90 and position GK (Goal Keeper)
Sergio Ramos with score: 88 and position CB (Center Back)
M. Verratti with score: 87 and position CM (Center Midfielder)
A. Hakimi with score: 85 and position RB (Right Back)
Luis Alberto with score: 84 and position CM (Center Midfielder)
L. Digne with score: 84 and position LB (Left Back)
M. Ginter with score: 84 and position CB (Center Back)
Y. Carrasco with score: 84 and position LM (Left Midfielder)
L. SanÃ© with score: 84 and position RM (Right Midfielder)

The substitutes are: 
M. Sabitzer with score: 84 and position CM (Center Midfielder)
F. Kostic with score: 84 and position LM (Left Midfielder)
R. Guerreiro with score: 84 and position LB (Left Back)
Ricardo Pereira with score: 84 and position RB (Right Back)
M. Maignan with score: 84 and position GK (Goal Keeper)
M. Brozovic with scor

C:\Users\ximen\AppData\Local\Temp\ipykernel_15664\2875568670.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_team['Starting_11'] = selected_team['sofifa_id'].isin(starting_11).astype(int)


In [28]:
summary = selected_team.iloc[:,[1,2,3,4,5,6,9,10,11,-5,-4,-3,-2]]
init_11 = selected_team.iloc[list(selected_team.Starting_11==1),[1,2,3,4,5,6,9,11,-5,-4,-3,-2]]
sust_12 = selected_team.iloc[list(selected_team.Starting_11==0),[1,2,3,4,5,6,9,11,-5,-4,-3,-2]]

In [29]:
display(init_11)

,short_name,score,value_eur,annual_wage_eur,potential,age,nationality_name,height_cm,Potential_Growth,Binary_Potential,Binary_Age,Standard_Position
0,L. Messi,93,78000000.0,16640000.0,93,34,Argentina,170,0,0,1,RW (Right Wing)
2,Cristiano Ronaldo,91,45000000.0,14040000.0,91,36,Portugal,187,0,0,1,CF (Center Forward)
7,M. Neuer,90,13500000.0,4472000.0,90,35,Germany,193,0,0,1,GK (Goal Keeper)
22,Sergio Ramos,88,24000000.0,5980000.0,88,35,Spain,184,0,0,1,CB (Center Back)
38,M. Verratti,87,79500000.0,8060000.0,87,28,Italy,165,0,0,0,CM (Center Midfielder)
95,A. Hakimi,85,69500000.0,5200000.0,88,22,Morocco,181,3,1,0,RB (Right Back)
108,Luis Alberto,84,42500000.0,4940000.0,84,28,Spain,183,0,0,0,CM (Center Midfielder)
111,L. Digne,84,40500000.0,5720000.0,84,27,France,178,0,0,0,LB (Left Back)
119,M. Ginter,84,42500000.0,2340000.0,85,27,Germany,191,1,0,0,CB (Center Back)
121,Y. Carrasco,84,45000000.0,4212000.0,84,27,Belgium,185,0,0,0,LM (Left Midfielder)


In [30]:
display(sust_12)

,short_name,score,value_eur,annual_wage_eur,potential,age,nationality_name,height_cm,Potential_Growth,Binary_Potential,Binary_Age,Standard_Position
116,M. Sabitzer,84,48000000.0,5720000.0,85,27,Austria,178,1,0,0,CM (Center Midfielder)
122,F. Kostic,84,42500000.0,2704000.0,84,28,Serbia,184,0,0,0,LM (Left Midfielder)
124,R. Guerreiro,84,40500000.0,4108000.0,84,27,Portugal,170,0,0,0,LB (Left Back)
125,Ricardo Pereira,84,40500000.0,6760000.0,84,27,Portugal,175,0,0,0,RB (Right Back)
127,M. Maignan,84,49500000.0,2340000.0,89,25,France,191,5,1,0,GK (Goal Keeper)
128,M. Brozovic,84,37000000.0,6760000.0,84,28,Croatia,181,0,0,0,CB (Center Back)
130,J. GimÃ©nez,84,52000000.0,3848000.0,88,26,Uruguay,185,4,1,0,CB (Center Back)
131,N. Fekir,84,45000000.0,2184000.0,84,27,France,173,0,0,0,CM (Center Midfielder)
150,Muniain,83,36500000.0,2288000.0,83,28,Spain,170,0,0,0,CM (Center Midfielder)
162,K. Volland,83,37000000.0,4316000.0,83,28,Germany,179,0,0,0,CF (Center Forward)


In [31]:
# To see that the model meet the criteria the limit of non European players:
non_european = []
for i in range (len(summary)):
    if summary.iloc[i,7]==0:
        non_european.append(summary.iloc[i,0])
print("The non European Union players are:",non_european)
print("The total number of players that are not from a European Union country are:",len(non_european),"vs the limit of 5")

The non European Union players are: ['L. Messi', 'A. Hakimi', 'F. Kostic', 'J. GimÃ©nez', 'L. Ocampos']
The total number of players that are not from a European Union country are: 5 vs the limit of 5


In [32]:
# To see that the model meet the criteria the limit of non budget:
if response_1 == "a":
    x=Low_budget
elif response_1 == "b":
    x=Medium_budget
elif response_1 == "c":
    x=High_budget

print("The total value plus wage of the team is: ""{:,}".format(summary.iloc[:,2].sum()+summary.iloc[:,3].sum())
      ,"vs the limit of:","{:,}".format(x))

The total value plus wage of the team is: 1,156,128,000.0 vs the limit of: 1,171,276,000


In [33]:
# To see that the model meet the criteria the constraint of have at least one defense higher than 186 cm:
tall_def = []
for i in range (len(summary)):
    if summary.iloc[i,8]>=186 and (summary.iloc[i,-1]==("CB (Center Back)") 
                               or (summary.iloc[i,-1]==("LB (Left Back)")
                               or (summary.iloc[i,-1]==("RB (Right Back)")))):
        tall_def.append(summary.iloc[i,0])
print("The defense players higher than 186 cm are:",tall_def)
print("The total number of players higher than 186 cm are:",len(tall_def),"vs the constrait of at least 1")

The defense players higher than 186 cm are: ['M. Ginter']
The total number of players higher than 186 cm are: 1 vs the constrait of at least 1


In [34]:
# To see that the model meet the criteria that at least 2 but no more thatn 4 players are older than 29 years old:
exp_players = []
for i in range (len(summary)):
    if summary.iloc[i,5]>29:
        exp_players.append(summary.iloc[i,0])
print("The players older than 29 years are:",exp_players)
print("The total number of players older than 29 years are:",len(exp_players),"vs the range from 2 to 4")

The players older than 29 years are: ['L. Messi', 'Cristiano Ronaldo', 'M. Neuer', 'Sergio Ramos']
The total number of players older than 29 years are: 4 vs the range from 2 to 4


In [35]:
# To see that the model meet the criteria that at least 4 players meet the potential growth criteria:
pot_players = []
for i in range (len(summary)):
    if summary.iloc[i,10]==1:
        pot_players.append(summary.iloc[i,0])
print("The players that meet potential growth are:",pot_players)
print("The total number of players that meet potential growth are:",len(pot_players),"vs the criteria of at least 4")

The players that meet potential growth are: ['A. Hakimi', 'M. Maignan', 'J. GimÃ©nez', 'L. SanÃ©']
The total number of players that meet potential growth are: 4 vs the criteria of at least 4


In [36]:
# To see that the model meet the criteria that at least 1 Goal Keeper meets the potential growth criteria:
pot_gk = []
for i in range (len(summary)):
    if summary.iloc[i,10]==1 and summary.iloc[i,-1]==("GK (Goal Keeper)"):
        pot_gk.append(summary.iloc[i,0])
print("The Goal Keeper that meets the potential growth is:",pot_gk)

The Goal Keeper that meets the potential growth is: ['M. Maignan']
